<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/Working_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%capture
! apt-get install libboost-python1.71-dev openbabel libopenbabel-dev python-dev zlib1g-dev libeigen3-dev libgsl-dev
!rm -f Makefile*
!wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
!make clean
!make -j4

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
libboost-python1.71-dev is already the newest version (1.71.0-6ubuntu6).
libeigen3-dev is already the newest version (3.3.7-2).
libgsl-dev is already the newest version (2.5+dfsg-6build1).
libopenbabel-dev is already the newest version (3.0.0+dfsg-3ubuntu3).
openbabel is already the newest version (3.0.0+dfsg-3ubuntu3).
python-dev-is-python2 is already the newest version (2.7.17-4).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-2ubuntu1.5).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
--2023-04-05 18:05:02--  https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [2]:
!ls *

iMcLiBELa.h	 pyCOORD_MC.cpp  pyFindHB.so	 pyMol2.h      pyWRITER.cpp
lig.mol2.gz	 pyCOORD_MC.h	 pyGaussian.cpp  pyMol2.so     pyWRITER.h
lig.mol2.gz.1	 pyCOORD_MC.so	 pyGaussian.h	 pyPARSER.cpp  pyWRITER.so
Makefile	 pyEnergy2.cpp	 pyGaussian.so	 pyPARSER.h    rec.mol2.gz
McLiBELa.log	 pyEnergy2.h	 pyGrid.cpp	 pyPARSER.so   rec.mol2.gz.1
pyConformer.cpp  pyEnergy2.so	 pyGrid.h	 pyRAND.cpp
pyConformer.h	 pyFindHB.cpp	 pyGrid.so	 pyRAND.h
pyConformer.so	 pyFindHB.h	 pyMol2.cpp	 pyRAND.so

sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [3]:
%%capture
#!make clean
#!make
#!ls *.so

In [4]:
from pyPARSER import *
from pyMol2 import *
from pyWRITER import *
from pyGrid import *
from pyCOORD_MC import *
from pyFindHB import * 
from pyEnergy2 import *
from pyGaussian import *

In [7]:
Input = PARSER()
Input.output = "teste" #@param {type:"string"}
Input.scoring_function = 0 #@param {type:"integer"}
Input.use_grids = True #@param {type:"boolean"}
Input.grid_prefix = "McGrid" #@param {type:"string"}
Input.search_box_x = 30.0 #@param {type:"number"}
Input.search_box_y = 30.0 #@param {type:"number"}
Input.search_box_z = 30.0 #@param {type:"number"}
Input.grid_spacing = 0.3 #@param {type:"number"}
Input.x_dim = 8.0 #@param {type:"number"}
Input.y_dim = 8.0 #@param {type:"number"}
Input.z_dim = 8.0 #@param {type:"number"}
Input.write_grids = True;

In [8]:

Writer = WRITER(Input)
Coord = COORD_MC()
HB = FindHB()
lig = Mol2(Input, "lig.mol2.gz")
rec = Mol2(Input, "rec.mol2.gz")

for i in range(len(list(rec.residue_pointer))-1):
  HB.parse_residue(rec.residue_pointer[i]-1, rec.residue_pointer[i+1]-2, rec.resnames[i], rec, lig, 9.0)

HB.find_ligandHB("lig.mol2.gz", lig)

com = Coord.compute_com(lig)

Input.write_grids = True;
Grids = Grid(Input, Writer, rec, com)